# Look at WG met file (subset of WG L2 data file)



In [22]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib
import datetime as dt
################
# This allows us to import Tom_tools
import sys
sys.path.append('../Tom_tools/') # you may need to adjust this path
# sys.path.append('../SWOT_IW_SSH/jtf/Tom_tools/') # you may need to adjust this path
import Tom_tools_v1 as tt
################

In [2]:
# %matplotlib inline
%matplotlib qt 
savefig = True # set to true to save plots as file
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 400
plt.close('all')
plt.rcParams['axes.xmargin'] = 0


__figdir__ = './plots/'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
path = '/mnt/e/SMODE_data/pilot/WG/L2/'
WG = 'WHOI43'
file = WG+'_L2b_met.nc'

In [4]:
# This needs to be set before any dates are encoded as datetime64
# Because the time increment is so small, we get 'out of range' when starting from the standard datetime64 epoch of 1970
matplotlib.dates.set_epoch('2000-01-01T00:00:00') 

In [5]:
%%time
ds = xr.open_dataset(path+file, engine = 'netcdf4', decode_times = True) #decode_times = False, 

CPU times: user 544 ms, sys: 91.5 ms, total: 635 ms
Wall time: 1.01 s


In [6]:
ds

<xarray.Dataset>
Dimensions:                     (time_1Hz: 771961, time_10Hz: 7719601, time_15min: 858, wave_frequency: 153, time_20Hz: 15439201, Workhorse_time: 771980, Workhorse_range: 21, Nortek_Burst_Bin: 20, Nortek_Burst_Time: 5884631, Nortek_IBurst_Bin: 20, Nortek_IBurst_Time: 5884632)
Coordinates: (12/17)
  * time_1Hz                    (time_1Hz) datetime64[ns] 2021-10-28T00:59:59...
    latitude_1Hz                (time_1Hz) float64 ...
    longitude_1Hz               (time_1Hz) float64 ...
  * time_10Hz                   (time_10Hz) datetime64[ns] 2021-10-28T00:59:5...
    altitude_10Hz               (time_10Hz) float32 ...
  * time_15min                  (time_15min) datetime64[ns] 2021-10-28T00:59:...
    ...                          ...
  * Workhorse_time              (Workhorse_time) datetime64[ns] 2021-10-28T01...
  * Workhorse_range             (Workhorse_range) float32 20.0 nan ... nan nan
  * Nortek_Burst_Bin            (Nortek_Burst_Bin) uint8 1 2 3 4 ... 17 18 19 20
  * Nortek_Burst_Time           (Nortek_Burst_Time) datetime64[ns] 2021-10-28...
  * Nortek_IBurst_Bin           (Nortek_IBurst_Bin) uint8 1 2 3 4 ... 18 19 20
  * Nortek_IBurst_Time          (Nortek_IBurst_Time) datetime64[ns] 2021-10-2...
Data variables: (12/42)
    INST_RBR_Concerto           float64 ...
    INST_Gill                   float64 ...
    INST_SMP21                  float64 ...
    INST_SGR4                   float64 ...
    INST_UCTD                   float64 ...
    INST_WXT                    float64 ...
    ...                          ...
    wind_direction              (time_20Hz) float32 ...
    wind_vertical               (time_20Hz) float32 ...
    Workhorse_latitude          (Workhorse_time) float64 ...
    Workhorse_longitude         (Workhorse_time) float64 ...
    Workhorse_altitude          (Workhorse_time) float64 ...
    Nortek_IBurst_AHRS_GyroZ    (Nortek_IBurst_Time) int32 ...
Attributes: (12/39)
    DOI:                           10.5067/SMODE-GLID3
    keywords:                      EARTH SCIENCE > OCEANS > SALINITY/DENSITY ...
    keywords_vocabulary:           NASA Global Change Master Directory (GCMD)...
    Conventions:                   CF-1.8, ACDD-1.3
    id:                            PO.DAAC-SMODE-GLID3
    naming_authority:              gov.nasa
    ...                            ...
    geospatial_vertical_positive:  down
    title:                         S-MODE Pilot Field Campaign Fall 2021 Ocea...
    summary:                       Wave Glider WHOI43 was deployed by R/V Oce...
    uuid:                          631df6c2-9281-4646-a19f-457baa270127
    history:                       2022-03-14 Benjamin Greenwood - process in...
    date_created:                  2022-03-14T20:53:19Z

In [7]:
# Raw met plot from WG:
fig, axs = plt.subplots(4, 1, sharex=True)
fig.autofmt_xdate()
plt.subplot(4,1,1)
h1, = plt.plot(ds.time_1Hz, ds.WXT_air_temperature)
h2, = plt.plot(ds.time_1Hz, ds.UCTD_sea_water_temperature)
plt.legend([h1, h2],['Air temp.','SST'])
plt.ylabel('T [$^\circ$C]')
plt.title('Raw 1 Hz WXT measurements')

plt.subplot(4,1,2)
plt.plot(ds.time_1Hz, ds.WXT_rainfall_rate)
plt.ylabel('[mm/hr]')
plt.legend(['Precip. rate'])

plt.subplot(4,1,3)
plt.plot(ds.time_1Hz, ds.WXT_relative_humidity)
plt.ylabel('[%]')
plt.legend(['Rel. Humidity'])

plt.subplot(4,1,4)
# plt.plot(ds.time_20Hz, ds.wind_speed)
plt.plot(ds.time_1Hz, ds.WXT_wind_speed)
plt.ylabel('[m/s]')
plt.legend(['Wind speed'])



In [8]:
# There is a limited number of NaNs in WXT wind speed
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_1Hz, np.isnan(ds.WXT_wind_speed.values))
plt.title('NaNs in WXT wind speed')

Text(0.5, 1.0, 'NaNs in WXT wind speed')

In [9]:
# Let's interpolate them
ff = np.where(np.isnan(ds.WXT_wind_speed.values)==0)
t = ds.time_1Hz[ff]
wspd = ds.WXT_wind_speed[ff]
WXT_wspd_interp = np.interp(ds.time_1Hz, t, wspd)

In [10]:
'''
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

# plt.plot(t, np.isnan(wspd))
plt.plot(np.isnan(WXT_wspd_interp))
plt.title('NaNs in WXT wind speed after trying to remove nans')
''' # this checks out

"\nfig, axs = plt.subplots(1, 1)\nfig.autofmt_xdate()\n\n# plt.plot(t, np.isnan(wspd))\nplt.plot(np.isnan(WXT_wspd_interp))\nplt.title('NaNs in WXT wind speed after trying to remove nans')\n"

In [11]:
fig, axs = plt.subplots(1, 1)
tt.spectrum_band_avg(WXT_wspd_interp,1,75,winstr=None,plotflag=True,ebarflag=None)
plt.title('WXT wind speed')
plt.xlabel('Hz')
plt.ylabel('Spectral density [m$^2$/s$^2$/Hz]')

Text(0, 0.5, 'Spectral density [m$^2$/s$^2$/Hz]')

In [12]:
# There is also a limited number of NaNs in Gills wind speed
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_20Hz, np.isnan(ds.wind_speed.values))
plt.title('NaNs in Gill wind speed')

Text(0.5, 1.0, 'NaNs in Gill wind speed')

In [13]:
# Let's interpolate them
ff = np.where(np.isnan(ds.wind_speed.values)==0)
t = ds.time_20Hz[ff]
wspd = ds.wind_speed[ff]
wspd_interp = np.interp(ds.time_20Hz, t, wspd)

In [14]:
fig, axs = plt.subplots(1, 1)
tt.spectrum_band_avg(wspd_interp,1/20,115,winstr=None,plotflag=True,ebarflag=None)
plt.title('Gill wind speed')
plt.xlabel('Hz')
plt.ylabel('Spectral density [m$^2$/s$^2$/Hz]')

Text(0, 0.5, 'Spectral density [m$^2$/s$^2$/Hz]')

## There appears to be some residual wave contamination in the motion corected velocities, but it shouldn't be a big issue when we average to 1 min or longer

In [15]:
# Raw met plot from WG:
fig, axs = plt.subplots(4, 1, sharex=True)
fig.autofmt_xdate()
plt.subplot(4,1,1)
h1, = plt.plot(ds.time_1Hz, tt.run_avg1d(ds.WXT_air_temperature,15*60))
h2, = plt.plot(ds.time_1Hz, tt.run_avg1d(ds.UCTD_sea_water_temperature,15*60))
plt.legend([h1, h2],['Air temp.','SST'])
plt.ylabel('T [$^\circ$C]')
plt.title('15-min average WXT measurements')
plt.axis('tight')

plt.subplot(4,1,2)
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.WXT_rainfall_rate,15*60))
plt.ylabel('[mm/hr]')
plt.legend(['Precip. rate'])

plt.subplot(4,1,3)
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.WXT_relative_humidity,15*60))
plt.ylabel('[%]')
plt.legend(['Rel. Humidity'])

plt.subplot(4,1,4)
plt.plot(ds.time_20Hz-dt., tt.run_avg1d(ds.wind_speed,15*60*20))
plt.plot(ds.time_1Hz, tt.run_avg1d(ds.WXT_wind_speed,15*60))
plt.ylabel('[m/s]')
plt.legend(['Gill Wind speed','WXT wind speed'])

fig.tight_layout()

UFuncTypeError: Cannot cast ufunc 'subtract' input 1 from dtype('<m8[Y]') to dtype('<m8[ns]') with casting rule 'same_kind'

In [ ]:
plt.axis('tight')

In [25]:
ds.time_20Hz.values+np.timedelta64(20, 'Y')

UFuncTypeError: Cannot cast ufunc 'add' input 1 from dtype('<m8[Y]') to dtype('<m8[ns]') with casting rule 'same_kind'

In [20]:
type(ds.time_20Hz.values)

numpy.ndarray

In [ ]:
dt.datetime.